## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-12-16-52-00 +0000,wsj,Crypto Entrepreneur Do Kwon Pleads Guilty to F...,https://www.wsj.com/finance/currencies/crypto-...
1,2025-08-12-16-50-16 +0000,nypost,Chinese national in NYC tries to smuggle $1.4M...,https://nypost.com/2025/08/12/us-news/chinese-...
2,2025-08-12-16-49-04 +0000,startribune,Prosecutors dismiss case against teen accused ...,https://www.startribune.com/prosecutors-dismis...
3,2025-08-12-16-47-00 +0000,wsj,"Year-over-year inflation held steady in July, ...",https://www.wsj.com/economy/cpi-inflation-july...
4,2025-08-12-16-45-50 +0000,nyt,"When Trump Meets Putin, Anything Could Happen",https://www.nytimes.com/2025/08/12/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2298/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
45,trump,68
52,china,16
43,up,13
201,new,11
63,israel,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
260,2025-08-11-19-43-00 +0000,wsj,The Trump administration’s decision to charge ...,https://www.wsj.com/politics/national-security...,141
186,2025-08-12-01-47-00 +0000,wsj,President Trump said he would deploy National ...,https://www.wsj.com/politics/policy/trump-dc-n...,140
192,2025-08-12-01-00-00 +0000,wsj,President Trump’s decision to deploy National ...,https://www.wsj.com/politics/policy/trump-nati...,121
263,2025-08-11-19-33-26 +0000,nypost,Trump extends China tariff deadline for 90 day...,https://nypost.com/2025/08/11/world-news/trump...,118
236,2025-08-11-21-25-43 +0000,bbc,Trump says he will try to get back territory f...,https://www.bbc.com/news/articles/c0e99yqv332o...,112


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
260,141,2025-08-11-19-43-00 +0000,wsj,The Trump administration’s decision to charge ...,https://www.wsj.com/politics/national-security...
186,64,2025-08-12-01-47-00 +0000,wsj,President Trump said he would deploy National ...,https://www.wsj.com/politics/policy/trump-dc-n...
234,44,2025-08-11-21-33-29 +0000,nyt,Israel-Qatar Tensions Escalate After Israel Ki...,https://www.nytimes.com/2025/08/11/world/middl...
94,41,2025-08-12-12-00-00 +0000,nypost,Ex-NYPD cop claims Bill de Blasio ‘didn’t like...,https://nypost.com/2025/08/12/us-news/ex-nypd-...
214,36,2025-08-11-22-47-07 +0000,nyt,New York’s Democratic Governor Called in the T...,https://www.nytimes.com/2025/08/11/us/politics...
236,35,2025-08-11-21-25-43 +0000,bbc,Trump says he will try to get back territory f...,https://www.bbc.com/news/articles/c0e99yqv332o...
35,35,2025-08-12-15-45-00 +0000,wsj,The White House plans to conduct a far-reachin...,https://www.wsj.com/politics/policy/white-hous...
263,34,2025-08-11-19-33-26 +0000,nypost,Trump extends China tariff deadline for 90 day...,https://nypost.com/2025/08/11/world-news/trump...
232,32,2025-08-11-21-35-17 +0000,cbc,Canada-China water polo match paused briefly a...,https://www.cbc.ca/sports/olympics/summer/aqua...
0,31,2025-08-12-16-52-00 +0000,wsj,Crypto Entrepreneur Do Kwon Pleads Guilty to F...,https://www.wsj.com/finance/currencies/crypto-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
